## Embeddings and Neural Networks
### Embeddings
When we talk about making **embeddings**, we're talking about representing our data points or objects in our dataset as meaningful vectors. In graphs, and in natural language, we *could* represent nodes and words as sparse vectors. These might look like the sparse vectors we saw in the previous crappy example, where we ran k-means clustering on the adjacency matrix, or they might look like one-hot representations whose length is equal to the cardinality of a language's entire vocabulary. Of course, representations like these aren't very useful when we think about looking at questions of similarity or really any sort of machine learning applications. 

Enter **embeddings**. Embeddings can result from dimensionality reduction techniques like principal component analysis (PCA), but nowadays, when we think about embeddings, we tend to think about vectors that are created via training a neural network model to do a task. 

### Neural Networks
(this section borrows heavily from the coverage of ML in Bagrow & Ahn, section 16.2)

These can seem complicated to understand, but they, are at their heart, just doing a bunch of very parallelizable matrix operations. Here's a picture of a simple **feed-forward** neural network, sourced from this handy [Medium post](https://medium.com/analytics-vidhya/in-depth-explanation-of-feedforward-in-neural-network-mathematically-448092216b63). 

![feed-forward NN](https://miro.medium.com/v2/resize:fit:3200/1*ycDUAMaxDYaHx7xl9pqEjg.png)

Neural networks consist of sets of units (neurons) arranged in a sequence of layers. When we talk about the **architecture** of a neural network, we're talking about how the units are arranged, how many of them there are, how many layers we have, and the number of units per layer. In a fully connected feed-forward network, each unit in each layer is connected to all the nodes in the layer after it. First, the input nodes (the green ones) receive the input data (this could be attributes of a product, for example). Data moves through the network in the picture from left to right. The output of a particular unit is the activation function of the weighted sum of its inputs ($\sigma(\vec{w}^T \vec{x})$), where $\vec{w}$ is the weights of the inputs for that unit, and $\vec{x}$ is the inputs coming from the previous layer. $\sigma$ is the nonlinear activation function; in this case, we're probably thinking of the sigmoid function (pictured below & sourced [here](https://groups.csail.mit.edu/medg/hamish/medcomp3/sld007.htm)), but other activation functions, like $\tanh$, are possible as well (image sourced [here](https://sebastianraschka.com/faq/docs/activation-functions.html).

![sigmoid](https://groups.csail.mit.edu/medg/hamish/medcomp3/img007.gif)

![activations](https://sebastianraschka.com/images/faq/activation-functions/activation-functions.png)

The outputs we get from each unit in the middle layer then are passed to the final layer, and then we compare the final layer's output to our desired output. Maybe the $\vec{y}$ outputs in the picture are probabilities of the product belonging to a particular class, for example. Our task is to use a **loss function**, of which there are many, to express how far off we are from our correct answer. Once we've done that, we can use an algorithm called **backpropagation** to nudge the weights ($\vec{w}$) towards values that would be closer to producing the desired label. 

Our choice of loss function is important here; what works well for classifying products may be terrible for predicting flight prices, for example. For a more complete primer on loss functions, check out [this blog post](https://www.geeksforgeeks.org/loss-functions-in-deep-learning/). Generally, your loss function and the tasks you're setting up the neural network to learn will reflect what you want your neural network to do effectively. We might use a squared error loss function if we're predicting flight prices, and we might use a cross-entropy loss function to quantify how incorrect our predictions are for a classifier. There are more complicated loss functions and task setups, one of which we'll discuss in the next section. 

### \*tovec
#### Word2Vec
[Word2Vec](https://arxiv.org/pdf/1301.3781) was a pretty early natural language processing technique that was very exciting at the time. It trained a fairly simple neural network to do one of two tasks: Continuous Bag-of-Words (CBOW) or Skip-Gram. CBOW involves predicting which word is missing from a given context of surrounding words, and Skip-Gram involves doing the opposite: predicting the surrounding context given a specific word. The loss function here is cross-entropy loss; we use a softmax activation function to turn the outputs of the final layer into a probability distribution and then use cross-entropy loss to quantify how incorrect we were. Here's some more info; the links are where we sourced our images for [softmax](https://gombru.github.io/2018/05/23/cross_entropy_loss/) and [word2vec](https://devopedia.org/word2vec).

![word2vec tasks](https://devopedia.org/images/article/221/9279.1570465016.png)

![softmax + cross-entropy](https://gombru.github.io/assets/cross_entropy_loss/softmax_CE_pipeline.png) 


#### Node2Vec
[Node2Vec](https://arxiv.org/pdf/1607.00653) follows similar logic to Word2Vec. We do random walks through a network and treat these random walks like sentences. Then we can do CBOW or Skip-Gram to obtain node embeddings. At each iteration, Node2Vec samples $n * r$ random walks (with $r$ walks starting at each node). It has hyperparameters that influence how much we revisit nodes ($p$) and how much we favor visiting close nodes versus nodes that are further from where we started ($q$). 

When people around the time of Node2Vec were thinking about building node embeddings, they tended to consider homophily (nodes that are linked are likely to be similar) or structural equivalence (nodes with similarly structured neighborhoods are likely to be similar). Random walks that are closer to depth-first search tend to get further away from their start node, so they'll have a better sense of the community structure of a given graph and we'll get more information about node homophily (in theory). However, longer-range dependencies can be harder to make sense of and more complicated. If we instead do something more like a breadth-first search, we're thinking more about structural equivalence -- whose neighborhoods looks similar? This isn't likely to explore more distant parts of the graph, and nodes will repeat themselves a lot in a given walk. So Grover & Leskovec structured their random walks with hyperparameters $p$ and $q$ that allow us to figure out where on the BFS - DFS spectrum we want to end up. 

Our non-normalized probability $\alpha(v, x) * w(v, x)$ of visiting a given node $x$ after having visited $v$ in the previous step is:
$$
\alpha(v, x) = 
\begin{cases}
\frac{1}{p} & if d(v, x) = 0 \\
1 & if d(v, x) = 1 \\
\frac{1}{q} & if d(v, x) = 2
\end{cases}
$$

where $d(i, j)$ is the number of hops between node $i$ and node $j$. 

Here's an image from the original Node2Vec paper of the different outcomes of Node2Vec on a network generated from the novel *Les Misérables* with hyperparameter settings that lean more towards prioritizing homophily (closer to DFS) and structural equivalence (closer to BFS).

![lesmis](./images/les_mis_node2vec.png)

Node2Vec also incorporates **negative sampling**, where we only look at the outputs for a couple nodes that definitely **shouldn't** be in an input node's neighborhood. We make sure our outputs for those nodes are zero and do backpropagation accordingly; it's possible to tune how many negative samples we do for each positive sample. Adding negative samples improves our model's performance, and it's relatively cheap to do because we're not tuning all the weights in the model at once. 

We can do a version of Node2Vec where we're making embeddings for embeddings' sake, as we do in this setting, or we can try to train a classifier on Node2Vec embeddings to predict labels for nodes. Here we're going to train Node2Vec (without node or edge labels) on the Cambridge, MA network data. Let's see what happens! Our training code is borrowed from `pytorch-geometric` and can be found [here](https://github.com/pyg-team/pytorch_geometric/blob/master/examples/node2vec.py).

## On Training Neural Networks
We're going to use [`pytorch`](https://pytorch.org/) and [`pytorch-geometric`](https://pytorch-geometric.readthedocs.io/en/latest/) to make our neural network construction relatively pain-free. Pytorch is a widely used package for building & training neural networks. It's compatible with CUDA-capable GPUs (CUDA is Nvidia's deep learning framework that makes doing ML in GPUs possible). 

### GPUs
GPUs, or *graphical processing units*, are great at doing a lot of tiny calculations at the same time. This is different from CPUs, which are good at doing a few big calculations at once (this is a vast oversimplification, but it's good enough for an overview). Neural networks do a lot of vector and matrix multiplication (Remember those dot products from before? How do they scale up to a whole layer?) which is very easy to parallelize on a GPU. We're using GPUs today to speed up our Node2Vec training, which would be pretty slow on a CPU. 

### Train/Validation/Test Sets
A lot of the time, when we train a model (this goes for all models, not just neural network models!) we're worried about **overfitting**. Overfitting happens when your model learns the data it's trained on too well -- it's too accustomed to the training data and doesn't generalize well to new data. We might have too many parameters in our model, such that it's able to memorize the quirks of the training set too well, or we might've selected our training set improperly.

I once saw an example from a talk about explainability where scientists trained a model to guess where a Google StreetView photo was taken; it did really well on images taken at the same time as other images in the training set and really poorly on images taken at a later date. Turns out that the model was memorizing how windshield smudges changed over time and correlated the state of the windshield smudges to the car's location. So when the smudges changed (but the landscape didn't), the model was not able to identify the locations anymore. 

We usually split our data into 3 sets -- training, validation, and test. We use our **training** set to train the model; the **validation** set is used to figure out which choice of hyperparameters (like learning rate, optimizer, network size, etc.) works best for the dataset & model; and the **testing** set is for figuring out how well the finalized model does. The model **never** trains on the validation or testing data - it only updates itself based on the training data. A good rule of thumb is to use 10-15% of data for validation, 15-20% of the data for testing, and the rest for training. You should randomly split up your data such that the training, testing, and validation sets have similar distributions. 

Here, though, we're just trying to build embeddings to demonstrate Node2Vec's capabilities, so we're not holding out any data right now. Node2Vec *does* sample negative examples as it trains, so that the model learns which contexts or nodes are correct and builds accurate embeddings. If we were training a link prediction model with Node2Vec, as you can choose to do at the end of this class, we would at minimum hold out some fraction of links to make sure our embeddings were of good quality and useful for the downstream link prediction task. 

### Hyperparameters
**Hyperparameters** are parameters that control how we find the neural network's weights (the non-hyper parameters, or just plain parameters). Examples of hyperparameters are learning rate, number of layers, choice of optimizer, layer size, and activation function(s). Finding good hyperparameters can take a lot of time, depending on the number and types of hyperparameters you're playing with; platforms like [Weights and Biases](https://wandb.ai/site) have sprung up to help developers tune hyperparameters for their neural network models. 

## Appendix: Backpropagation (by request):
At a very high level, backpropagation is how we adjust our weights, going back from the output layer (and our loss function) all the way back to the weights from the input layer. 

It involves computing a bunch of partial derivatives (gradients) and adjusting our weights/biases (the parameters we're learning in our neural network) according to the relationship between the gradients and the loss. 

### What ingredients do we need to do backpropagation?

First, we need a loss function. Our loss function (or cost function) needs to be differentiable with respect to the weights and biases we use in the network. Our loss also has to be expressed as a function of the input and our weights and biases. For example, let's look at a toy example with one hidden layer and a mean squared error (MSE) loss function. 

![feed-forward NN](https://miro.medium.com/v2/resize:fit:3200/1*ycDUAMaxDYaHx7xl9pqEjg.png)

The simplified output $g(\vec{x})$ of our neural network with input $\vec{x}$, given weight matrices $W^{(1)}$ and $W^{(2)}$ and generic activation functions $\sigma^{(1)}$ and $\sigma^{(2)}$, is

$$g(x) = \sigma^{(2)}(W^{(2)} \sigma^{(1)}(W^{(1)} \vec{x}))$$

Our loss function, with ground truth $\vec{z}$, is $\lvert \lvert g(\vec{x}) - \vec{z} \rvert \rvert$. In the generalized sense, we can use a generic loss function $C(g(\vec{x}), \vec{z})$. 

Next, we need partial derivatives and the chain rule!

This is how backpropagation adjusts the weights -- we compute the partial derivative of our cost function $C$ with respect to one weight from node $i$ to node $j$ in the $2^{nd}$ matrix of weights:

$$\frac{\delta C}{\delta w^{(2)}_{ij}} = \frac{\delta C}{\delta y_j} \frac{\delta y_j}{\delta w^{(2)}_{ij}}$$

Here, $y_j$ is the $j^{th}$ output of our network (in the output layer).

$$y_j = \sigma^{(2)}(\sum_{i}w^{(2)}_{ij} * h_i)$$

In other words, we're passing the dot product of row $j$ of $W^{(2)}$ and $\vec{h}$, our hidden layer's output, through a sigmoid function. Let's call $\sum_{i}w^{(2)}_{ij} * h_i$ $o_j$, and let's expand our partial derivative expression using the chain rule once more. 

$$\frac{\delta C}{\delta w^{(2)}_{ij}} = \frac{\delta C}{\delta y_j} \frac{\delta y_j}{o_j} \frac{o_j}{\delta w^{(2)}_{ij}}$$

What are we doing here? We're tracing how our specific weight $w^{(2)}_{ij}$ affects our computed loss for a particular input (or batch of inputs). 

We know that $\frac{\delta y_j}{o_j}$ is the partial derivative of the activation function $\sigma^{(2)}$. 

Additionally, we know that $\frac{o_j}{\delta w^{(2)}_{ij}}$ is 

$$\frac{\delta}{\delta w^{(2)}_{ij}}\sum_{k}w^{(2)}_{kj}h_k$$

Only one term in this sum relies on $w^{(2)}_{ij}$ -- that's $w^{(2)}_{ij} h_i$. This means this part of our partial derivative reduces to  

$$\frac{o_j}{\delta w^{(2)}_{ij}} = h_i$$

Now let's look at $\frac{\delta y_j}{h_j}$. Let's say we're using a sigmoid activation function; in this case, this part of our partial derivative is 

$$\frac{\delta}{\delta h_j}\sigma(h_j) = \sigma(h_j) (1 - \sigma(h_j)) = y_j * (1 - y_j)$$

If we're using MSE for the loss function $C$ and $\vec{z}$ is our ground truth answer, 

$$\frac{\delta C}{\delta y_j} = 2 (z_j - y_j)$$.

Therefore, the gradient of our loss with respect to $w^{(2)}_{ij}$ is 

$$\frac{\delta C}{\delta w^{(2)}_{ij}} = 2 (z_j - y_j) * y_j * (1 - y_j) * h_i$$.

### Moving right along/TL;DR (For those who hate math!!)
We take partial derivatives with the chain rule to figure out how much our loss function changes with respect to a particular parameter (like a weight or bias) in the neural network. 

Then we can change that specific weight with this information. We usually have a learning rate $\eta$ (or an optimizer that governs the learning rate, which is fancier) that tells us how much to change a weight/bias with respect to our computed gradient.

$$\delta w^{(2)}_{ij} = \eta \frac{\delta C}{\delta w^{(2)}_{ij}}$$.

We don't want to update our parameters too much based on any one example, which is why the learning rate tends to be pretty small (much less than 1) and optimizers will lower the learning rate as training goes on and the model gets better at its task.

Let's review how backpropagation works by watching [this video](https://www.youtube.com/watch?v=GlcnxUlrtek&list=PLiaHhY2iBX9hdHaRr6b7XevZtgZRa1PoU&index=4). 